**Importing the necessary libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Loading Data**

In [4]:
data = pd.read_csv(r'C:\Users\rehma\Pictures\Daily_tasks\Rahman-Analytics-Lab\data.csv', encoding="ISO-8859-1") 
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
data.shape

(541909, 8)

### Data processing

**Missing data**

In [6]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

CustomerID 135080 of missing rows (24.93% missing values)

The CustomerID column contains nearly a quarter of missing data. This column is essential for clustering customers,Imputing such a large percentage of missing values might introduce significant bias or noise into the analysis. By removing rows with missing values in the CustomerID and Description columns, we aim to construct a cleaner and more reliable dataset

In [7]:
data = data[data['CustomerID'].isna()==False]

In [8]:
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

**Duplicated Rows**

In [9]:
data.duplicated().sum()

5225

In [10]:
data.drop_duplicates(inplace= True)

**Converting dtypes**

In [11]:
data.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

let's convert 'InvoiceDate' to a datetime data type and 'CustomerID' to an integer data type.

In [12]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [13]:
data['CustomerID'] = data['CustomerID'].astype(int , errors = 'ignore')

**Treating With Cancelled Transactions**

we will drop it now and and analysis it later

In [14]:
data = data[data['InvoiceNo'].apply(lambda x : x[0]) != 'C']

## Customer Lifetime Value

There are lots of approaches available for calculating CLTV. Everyone has his own view on it that suitable for his bussines

__CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.__

In [17]:
#Calulate total purchase
data['total purchase'] = data['UnitPrice'] * data['Quantity']

In [18]:
data_group = data.groupby('CustomerID' , as_index=False).agg({
    'total purchase':'sum' , 'InvoiceNo':'count' ,'Country':'min' ,'InvoiceDate':lambda date:(max(date) - min(date)).days}).rename(
      columns = {'InvoiceNo' : 'num_transactions' , 'InvoiceDate':'days'})

__1. Calculate Average Order Value__

In [19]:
data_group['avg_order_value'] = data_group['total purchase']/data_group['num_transactions']

In [20]:
data_group.head()

,CustomerID,total purchase,num_transactions,Country,days,avg_order_value
0,12346,77183.60,1,United Kingdom,0,77183.600000
1,12347,4310.00,182,Iceland,365,23.681319
2,12348,1797.24,31,Finland,282,57.975484
3,12349,1757.55,73,Italy,0,24.076027
4,12350,334.40,17,Norway,0,19.670588


__2. Calculate Purchase Frequency__

In [21]:
purchase_frequency=sum(data_group['num_transactions'])/data_group.shape[0]

__3. Calculate Repeat Rate and Churn Rate__

In [22]:
repeat_rate = len(data_group[data_group['num_transactions'] >= 2]) / len(data_group)
round(repeat_rate*100.0 , 2)

98.34

the percentage of repeat_rate is :98.34 %

__Churn Rate__

In [23]:
#Churn Rate
churn_rate=1-repeat_rate
round(churn_rate*100.0 , 2)

1.66

the percentage of Churn rate is :1.66 %